In [1]:
import transformers
import torch
from huggingface_hub import notebook_login

notebook_login()


In [2]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
401 Client Error. (Request ID: Root=1-6707f89d-7d5ca8c5282e57a47d1c3a09;6d4c1498-792c-49f2-a335-250baf930dd2)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
import json

output_store = []
fragment_indecies = []
# Открываем jsonl файл для чтения
with open('/kaggle/input/semeval-2025-task-3-mu-shroom-dataset/Dataset/Validation/mushroom.en-val.v2.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        # Преобразуем строку в json объект
        data = json.loads(line.strip())
        
        # Формируем строку в нужном формате
        formatted_str = f'"query":"{data["model_input"]}"\n"model_output":"{data["model_output_text"]}"'
        
        messages = [
            {"role": "system", "content": "You are a fact-checking assistant. Your task is to identify fragments of the response that are hallucinations – parts of the text that are factually incorrect or made up by model. Pay attention to facts, dates, numbers, places. Detect only hallucination words, without neighbour words. Give me only a list of fragments-hallucinations you found in model output."},
            {"role": "user", "content": formatted_str},
        ]

        outputs = pipeline(
            messages,
            max_new_tokens=256,
            temperature=0.1
        )
        output = outputs[0]["generated_text"][-1]['content']
        fragment_indecies = []
        for hall in [item[item.find(' '):].strip() for item in output.split('\n') if item]:
            fragment_indecies.append([data["model_output_text"].find(hall), 
                                      data["model_output_text"].find(hall) + len(hall)])
        output_store.append(output)
        y_pred.append(fragment_indecies)
        
        print(outputs[0]["generated_text"][-1])
        print(y_pred)


In [ ]:
import json
data_pred = []
# Открываем jsonl файл для чтения
with open('/kaggle/input/semeval-2025-task-3-mu-shroom-dataset/Dataset/Validation/mushroom.en-val.v2.jsonl', 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        item = json.loads(line)
        item['soft_labels'] = [{'start': y_pred[i][j][0], 'prob': 1, 'end': y_pred[i][j][1]}
                               for j in range(len(y_pred[i]))]
        item['hard_labels'] = [y_pred[i][j] for j in range(len(y_pred[i]))]
        data_pred.append(item)
        
data_pred

In [ ]:
import json
data_true = []
# Открываем jsonl файл для чтения
with open('/kaggle/input/semeval-2025-task-3-mu-shroom-dataset/Dataset/Validation/mushroom.en-val.v2.jsonl', 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        data_true.append(json.loads(line))
        
data_true

In [ ]:
ious = []

for i in range(50):
    data1 = data_true[i]
    data2 = data_pred[i]
    ious.append(score_iou(data1, data2))
print(np.mean(ious))

In [ ]:
ious

In [ ]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np

import argparse as ap

def recompute_hard_labels(soft_labels):
    """optionally, infer hard labels from the soft labels provided"""
    hard_labels = [] 
    prev_end = -1
    for start, end in (
        (lbl['start'], lbl['end']) 
        for lbl in sorted(soft_labels, key=lambda span: (span['start'], span['end']))
        if lbl['prob'] > 0.5
    ):
        if start == prev_end:
            hard_labels[-1][-1] = end
        else:
            hard_labels.append([start, end])
        prev_end = end
    return hard_labels

def load_jsonl_file_to_records(filename):
    """read data from a JSONL file and format that as a `pandas.DataFrame`. 
    Performs minor format checks (ensures that soft_labels are present, optionally compute hard_labels on the fly)."""
    df = pd.read_json(filename, lines=True)
    if 'hard_labels' not in df.columns:
        df['hard_labels'] = df.soft_labels.apply(recompute_hard_labels)
    # adding an extra column for convenience
    df['text_len'] = df.model_output_text.apply(len)
    df = df[['id', 'soft_labels', 'hard_labels', 'text_len']]
    return df.sort_values('id').to_dict(orient='records')

def score_iou(ref_dict, pred_dict):
    """computes intersection-over-union between reference and predicted hard labels, for a single datapoint.
    inputs:
    - ref_dict: a gold reference datapoint,
    - pred_dict: a model's prediction
    returns:
    the IoU, or 1.0 if neither the reference nor the prediction contain hallucinations
    """
    # ensure the prediction is correctly matched to its reference
    assert ref_dict['id'] == pred_dict['id']
    # convert annotations to sets of indices
    ref_indices = {idx for span in ref_dict['hard_labels'] for idx in range(*span)}
    pred_indices = {idx for span in pred_dict['hard_labels'] for idx in range(*span)}
    # avoid division by zero
    if not pred_indices and not ref_indices: return 1.
    # otherwise compute & return IoU
    return len(ref_indices & pred_indices) / len(ref_indices | pred_indices)

def score_cor(ref_dict, pred_dict):
    """computes Spearman correlation between predicted and reference soft labels, for a single datapoint.
    inputs:
    - ref_dict: a gold reference datapoint,
    - pred_dict: a model's prediction
    returns:
    the Spearman correlation, or a binarized exact match (0.0 or 1.0) if the reference or prediction contains no variation
    """
    # ensure the prediction is correctly matched to its reference
    assert ref_dict['id'] == pred_dict['id']
    # convert annotations to vectors of observations
    ref_vec = [0.] * ref_dict['text_len']
    pred_vec = [0.] * ref_dict['text_len']
    for span in ref_dict['soft_labels']:
        for idx in range(span['start'], span['end']):
            ref_vec[idx] = span['prob']
    for span in pred_dict['soft_labels']:
        for idx in range(span['start'], span['end']):
            pred_vec[idx] = span['prob']
    # constant series (i.e., no hallucination) => cor is undef
    if len({round(flt, 8) for flt in pred_vec}) == 1 or len({round(flt, 8) for flt in ref_vec}) == 1 : 
        return float(len({round(flt, 8) for flt in ref_vec}) == len({round(flt, 8) for flt in pred_vec}))
    # otherwise compute Spearman's rho
    return spearmanr(ref_vec, pred_vec).correlation

def main(ref_dicts, pred_dicts, output_file=None):
    assert len(ref_dicts) == len(pred_dicts)
    ious = np.array([score_iou(r, d) for r, d in zip(ref_dicts, pred_dicts)])
    cors = np.array([score_cor(r, d) for r, d in zip(ref_dicts, pred_dicts)])
    if output_file is not None:
        with open(output_file, 'w') as ostr:
            print(f'IoU: {ious.mean():.8f}', file=ostr)
            print(f'Cor: {cors.mean():.8f}', file=ostr)
    return ious, cors

if False:
    p = ap.ArgumentParser()
    p.add_argument('ref_file', type=load_jsonl_file_to_records)
    p.add_argument('pred_file', type=load_jsonl_file_to_records)
    p.add_argument('output_file', type=str)
    a = p.parse_args()
    print(a)
    _ = main(a.ref_file, a.pred_file, a.output_file)



In [ ]:
messages = [
    {"role": "system", "content": "You are a fact-checking assistant. Your task is to identify fragments of the response that are hallucinations – parts of the text that are factually incorrect or made up by model. Pay attention to dates, numbers, places. Detect only hallucination words, without neighbour words. Give me only a list of fragments-hallucinations you found in model output."},
    {"role": "user", "content": """
    "query":"How many genera does the Erysiphales order contain?"\n"model_output_text":"The Elysiphale order contains 5 genera."
    """},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    temperature=0.1
)
print(outputs[0]["generated_text"][-1])

In [ ]:
messages = [
    {"role": "system", "content": "You are a fact-checking assistant. Your task is to identify fragments of the response that are hallucinations – parts of the text that are factually incorrect or made up by the model. Detect only hallucination words, without neighbour words. Give me only a list of fragments-hallucinations you found in 'model_output'."},
    {"role": "user", "content": """
    "query":"When did Chance the Rapper debut?"\n"model_output_text":"Chance the rapper debuted in 2011."
    """},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    temperature=0.1
)
print(outputs[0]["generated_text"][-1])

In [ ]:
messages = [
    {"role": "system", "content": "You are a fact-checking assistant. Your task is to identify fragments of the response that are hallucinations – parts of the text that are factually incorrect or made up by the model. Detect only hallucination words, without neighbour words. Give me only a list of fragments-hallucinations you found in 'model_output'."},
    {"role": "user", "content": """
    "query":"If today is 14th October, and it is not a leap year, how many days remain until the end of the year?"\n"model_output_text":"There are 365 days in a year. So, if today's date is October 13th, then there are only 162 days left until December 31st."
    """},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

In [ ]:
messages = [
    {"role": "system", "content": "You are a fact-checking assistant. Your task is to identify fragments of the response that are hallucinations – parts of the text that are factually incorrect or made up by the model. Detect only hallucination words, without neighbour words. Give me only a list of fragments-hallucinations you found in 'model_output'."},
    {"role": "user", "content": """
    "query":"Which network released the TV series of the The Punisher?"\n"model_output_text":"The Puncher network that released this TV show is Netflix."
    """},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

In [ ]:
messages = [
    {"role": "system", "content": "You are a fact-checking assistant. Your task is to identify fragments of the response that are hallucinations – parts of the text that are factually incorrect or made up by model. Pay attention to dates, numbers, places. Detect only hallucination words, without neighbour words. Give me only a list of fragments-hallucinations you found in model output."},
    {"role": "user", "content": """
    "query":"When was Bischofsheim a constitutent community of Mainz?"\n"model_output_text":" Bishofshain was a constituent community in the early Middle Ages.\n"
    """},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    temperature=0.1
)
print(outputs[0]["generated_text"][-1])

In [ ]:
messages = [
    {"role": "system", "content": "You are a fact-checking assistant. Your task is to identify fragments of the response that are hallucinations – parts of the text that are factually incorrect or made up by the model. Detect only hallucination words, without neighbour words. Give me only a list of fragments-hallucinations you found in 'model_output_text'. Pay attention to the facts, not to literacy and typos."},
    {"role": "user", "content": """
    query:"Who first described the white-winged chough?"\nmodel_output:"The white winged cough was first discovered by the English naturalist John Latham in 1770."
    """},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    temperature=0.1
)
print(outputs[0]["generated_text"][-1])